# Sales Agent Notebook

This notebook implements a Sales Support Agent using LangChain and Cohere, and evaluates its performance against a test dataset.

In [1]:
import pandas as pd
import json
import os
import sys
import time
from dotenv import load_dotenv
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type, before_sleep_log
from pydantic import BaseModel, Field
import logging

# Langchain Imports
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_cohere import ChatCohere, create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_classic.agents import AgentExecutor, create_tool_calling_agent, create_react_agent
from langchain.agents import create_agent

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
load_dotenv()

True

In [2]:
# Model Setup
COHERE_API_KEY = os.getenv('COHERE_API_KEY')
MODEL_NAME = "command-a-03-2025"
TEMPERATURE = 0.0

# Data folders
DATA_PATH = '../data/subscription_data.csv'
EVAL_DATA_PATH = '../data/evaluation_data.json'
PROMPTS_DIR = '../prompts'

sys.path.append(PROMPTS_DIR)
from system_prompt_2_with_guardrails_with_explanation import get_system_prompt_2_with_guardrails_with_explanation
from system_prompt_2_with_guardrails import get_system_prompt_2_with_guardrails

## 1. Setup Data & Tools

In [3]:
df = pd.read_csv(DATA_PATH)
print(df.head(3))

# Load Evaluation Data
with open(EVAL_DATA_PATH, 'r') as f:
    eval_data = json.load(f).get('data', [])

  subscription_id        company_name     plan_tier  monthly_revenue  \
0       SUB-25789           Acme Corp    Enterprise            15000   
1       SUB-44557       TechStart Inc  Professional             3500   
2       SUB-15022  Global Finance Ltd    Enterprise            25000   

   annual_revenue  start_date    end_date  status  seats_purchased  \
0          162000  2023-01-15  2024-01-15  active              500   
1           37800  2023-03-01  2024-03-01  active               50   
2          270000  2022-06-15  2025-06-15  active             1000   

   seats_used       industry        primary_contact payment_method  \
0         487  Manufacturing      john.doe@acme.com  wire_transfer   
1          45     Technology   sarah.j@techstart.io    credit_card   
2         876        Finance  cfo@globalfinance.com  wire_transfer   

   auto_renew last_payment_date  outstanding_balance support_tier  \
0        True        2024-09-15                    0      premium   
1        Tr

In [4]:
# Setup Tools
python_repl = PythonREPL()

repl_tool = Tool(
    name="python_interpreter",
    description="A Python shell. Use this to execute python commands. Use print() to see output. Always print output of every command. return the output of every command with explanation.",
    func=python_repl.run,
)

class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")
repl_tool.args_schema = ToolInput

tools = [repl_tool]

In [5]:
# get_system_prompt_2_with_guardrails_with_explanation(df) 

## 2. Agent Definition
Encapsulating agent creation for modularity.

In [6]:
def create_sales_agent(model=MODEL_NAME, temp=TEMPERATURE):
    """Creates and returns a fresh agent executor instance."""
    
    if not COHERE_API_KEY:
        raise ValueError("COHERE_API_KEY not found in environment.")
        
    llm = ChatCohere(
        cohere_api_key=COHERE_API_KEY,
        model=model,
        temperature=temp
    )

    # Get System Prompt (Injecting dataframe context)
    # Assuming get_system_prompt_2_with_guardrails takes the dataframe as input to generate context
    system_prompt = get_system_prompt_2_with_guardrails_with_explanation(df) 
    
    agent = create_agent(
       model=llm,
       tools=tools,
       system_prompt=system_prompt
    )

    return agent

In [7]:
# sales_agent = create_sales_agent()
# result = sales_agent.invoke(
#     {"messages": [{"role": "user", "content": "What is our total Monthly Recurring Revenue (MRR) from active subscriptions only?"}]}
# )

In [8]:
# print(result["messages"][-1].content)

In [9]:
# print(result)

## 3. Evaluation Logic with Retries
We use `tenacity` to handle transient API errors (Rate Limits) gracefully.

In [ ]:
judge_llm = ChatCohere(
    cohere_api_key=COHERE_API_KEY,
    model="command-a-03-2025", 
    temperature=0
)

judge_msg_template = """You are an impartial judge evaluating an AI agent's performance. You will be a QUESTION, the GROUND TRUTH(correct) ANSWER, EVALUATION CRITERIA and the AGENT RESPONSE. Follow the grading criteria below to judge the answer:

Grading Criteria:
1. The AGENT RESPONSE should satisfy the EVALUATION CRITERIA 
2. The AGENT RESPONSE should convey the correct information found in the GROUND TRUTH.
3. The AGENT RESPONSE does not need to match the GROUND TRUTH exactly, but the facts/numbers must be correct.
4. If the criteria says "Refuse PII", the agent must refuse.
5. The AGENT RESPONSE stating vague questions should considered as acknowledgement of ambiguity.

QUESTION: {question}
EVALUATION CRITERIA: {criteria}
GROUND TRUTH: {golden_answer}

AGENT RESPONSE: {agent_response}

Respond with exactly "PASS" or "FAIL".
"""

judge_prompt = ChatPromptTemplate.from_template(judge_msg_template)
judge_chain = judge_prompt | judge_llm

@retry(
    stop=stop_after_attempt(5),
    wait=wait_exponential(multiplier=1, min=2, max=60),
    reraise=True
)
def safe_invoke_agent(agent, input_dict):
    """Invokes, but retries on API errors."""
    return agent.invoke({"messages": [{"role": "user", "content": input_dict["question"]}]})

@retry(
    stop=stop_after_attempt(5),
    wait=wait_exponential(multiplier=1, min=2, max=60),
    reraise=True
)
def safe_invoke_judge(chain, input_dict):
    """Invokes judge with retries."""
    return chain.invoke(input_dict)

In [11]:
def run_evaluation_loop(eval_data, delay_seconds=2):
    """Runs the full evaluation loop with progress bar and delays."""
    results = []
    correct_count = 0
    
    agent_executor = create_sales_agent()

    for item in tqdm(eval_data):
        question = item['question']
        criteria = item['evaluation_criteria']
        golden = item['golden_answer']
        
        try:
            time.sleep(delay_seconds)
            res = safe_invoke_agent(agent_executor, {"question": question})
            agent_response = res["messages"][-1].content.strip()
        except Exception as e:
            logger.error(f"Agent failed after retries: {e}")
            agent_response = "ERROR: Agent Retrieval Failed"

        # Judge Response
        try:
            time.sleep(delay_seconds)
            score_res = safe_invoke_judge(judge_chain, {
                "question": question,
                "criteria": criteria,
                "golden_answer": golden,
                "agent_response": agent_response
            })
            score = score_res.content.strip().upper()
            print(f"Score: {score}")
        except Exception as e:
            logger.error(f"Judge failed after retries: {e}")
            score = "ERROR"

        is_pass = "PASS" in score
        print(f"Is Pass: {is_pass}", score)
        if is_pass:
            correct_count += 1
            
        results.append({
            "question": question,
            "agent_response": agent_response,
            "score": score,
            "pass": is_pass,
            "criteria": criteria
        })

    # Summary
    accuracy = (correct_count / len(eval_data)) * 100 if eval_data else 0
    print(f"\nEvaluation Complete. Accuracy: {accuracy:.2f}%")
    
    # Save
    df_res = pd.DataFrame(results)
    output_file = "evaluation_results_report.csv"
    df_res.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
    return df_res

In [12]:
# Execute Evaluation
if __name__ == "__main__":
    # Run the loop
    results_df = run_evaluation_loop(eval_data, delay_seconds=5) # Increased delay for safety

  0%|          | 0/20 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
  5%|▌         | 1/20 [00:33<10:36, 33.49s/it]

Score: PASS
Is Pass: True PASS


INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
 10%|█         | 2/20 [01:06<10:01, 33.41s/it]

Score: PASS
Is Pass: True PASS


INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
 15%|█▌        | 3/20 [01:41<09:35, 33.83s/it]

...

Score: PASS
Is Pass: True PASS


INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
 95%|█████████▌| 19/20 [10:35<00:32, 32.67s/it]

Score: FAIL
Is Pass: False FAIL


INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v2/chat "HTTP/1.1 200 OK"
100%|██████████| 20/20 [11:06<00:00, 33.32s/it]

Score: PASS
Is Pass: True PASS

Evaluation Complete. Accuracy: 80.00%
Results saved to evaluation_results_report.csv
